## Homework

In this homework, we'll deploy the Bank Marketing model from the homework 5.
We already have a docker image for this model - we'll use it for 
deploying the model to Kubernetes.


## Building the image
Clone the course repo if you haven't:

```
git clone https://github.com/DataTalksClub/machine-learning-zoomcamp.git
```

Go to the `course-zoomcamp/cohorts/2024/05-deployment/homework` folder and 
execute the following:


```bash
docker build -t zoomcamp-model:3.11.5-hw10 .

## Question 1

Run it to test that it's working locally:

```bash
docker run -it --rm -p 9696:9696 zoomcamp-model:3.11.5-hw10
```

And in another terminal, execute `q6_test.py` file:

```bash
python q6_test.py
```

You should see this:

```python
{'has_subscribed': True, 'has_subscribed_probability': <value>}
```

Here `<value>` is the probability of getting a subscription. You need to choose the right one.

In [2]:
!python q6_test.py

{'has_subscribed': True, 'has_subscribed_probability': 0.756743795240796}


## Installing `kubectl` and `kind`

You need to install:

* `kubectl` - https://kubernetes.io/docs/tasks/tools/ (you might already have it - check before installing)
* `kind` - https://kind.sigs.k8s.io/docs/user/quick-start/


In [1]:
!kind --version

kind version 0.26.0


In [2]:
!kind create cluster

Creating cluster "kind" ...
 ✓ Ensuring node image (kindest/node:v1.32.0) 🖼7l
 ✓ Preparing nodes 📦 7l
 ✓ Writing configuration 📜7l
 ✓ Starting control-plane 🕹️7l
 ✓ Installing CNI 🔌7l
 ✓ Installing StorageClass 💾7l
Set kubectl context to "kind-kind"
You can now use your cluster with:

kubectl cluster-info --context kind-kind

Have a question, bug, or feature request? Let us know! https://kind.sigs.k8s.io/#community 🙂


In [3]:
!kubectl cluster-info

Kubernetes control plane is running at https://127.0.0.1:33551
CoreDNS is running at https://127.0.0.1:33551/api/v1/namespaces/kube-system/services/kube-dns:dns/proxy

To further debug and diagnose cluster problems, use 'kubectl cluster-info dump'.


## Question 3

What's the smallest deployable computing unit that we can create and manage 
in Kubernetes (`kind` in our case)?

* Node
* Pod
* Deployment
* Service

The Pod is the smallest deployable unit in Kubernetes. It represents one or more containers sharing resources.

## Question 4

Now let's test if everything works. Use `kubectl` to get the list of running services.

What's the `Type` of the service that is already running there?

* NodePort
* ClusterIP
* ExternalName
* LoadBalancer


In [1]:
!kubectl get services

NAME         TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)   AGE
kubernetes   ClusterIP   10.96.0.1    <none>        443/TCP   17m


## Question 5

To be able to use the docker image we previously created (`zoomcamp-model:3.11.5-hw10`),
we need to register it with `kind`.

What's the command we need to run for that?

* `kind create cluster`
* `kind build node-image`
* `kind load docker-image`
* `kubectl apply`


In [1]:
! kind load docker-image svizor/zoomcamp-model:3.11.5-hw10

Image: "svizor/zoomcamp-model:3.11.5-hw10" with ID "sha256:020904a2523cec81c854a9b4679ae26d23f814e42350721136d1447f910a6b53" found to be already present on all nodes.


# Question 6

Now let's create a deployment config (e.g. `deployment.yaml`):

```yaml
apiVersion: apps/v1
kind: Deployment
metadata:
  name: subscription
spec:
  selector:
    matchLabels:
      app: subscription
  replicas: 1
  template:
    metadata:
      labels:
        app: subscription
    spec:
      containers:
      - name: subscription
        image: <Image>
        resources:
          requests:
            memory: "64Mi"
            cpu: "100m"            
          limits:
            memory: <Memory>
            cpu: <CPU>
        ports:
        - containerPort: <Port>
```

Replace `<Image>`, `<Memory>`, `<CPU>`, `<Port>` with the correct values.

What is the value for `<Port>`?

Apply this deployment using the appropriate command and get a list of running Pods. 
You can see one running Pod.

9696

In [7]:
!kubectl apply -f deployment.yaml

deployment.apps/subscription configured


In [8]:
!kubectl get deployment

NAME           READY   UP-TO-DATE   AVAILABLE   AGE
subscription   1/1     1            1           8m21s


## Question 7

Let's create a service for this deployment (`service.yaml`):

```yaml
apiVersion: v1
kind: Service
metadata:
  name: <Service name>
spec:
  type: LoadBalancer
  selector:
    app: <???>
  ports:
  - port: 80
    targetPort: <PORT>
```

Fill it in. What do we need to write instead of `<???>`?

Apply this config file.

subscription

In [9]:
!kubectl apply -f service.yaml

service/subscription configured


## Testing the service

We can test our service locally by forwarding the port 9696 on our computer 
to the port 80 on the service:

```bash
kubectl port-forward service/<Service name> 9696:80
```

Run `q6_test.py` (from the homework 5) once again to verify that everything is working. 
You should get the same result as in Question 1.

In [10]:
!kubectl port-forward service/subscription 9696:80

Forwarding from 127.0.0.1:9696 -> 9696
Forwarding from [::1]:9696 -> 9696
Handling connection for 9696
^C


In [11]:
!python q6_test.py

{'has_subscribed': True, 'has_subscribed_probability': 0.756743795240796}


## Autoscaling

Now we're going to use a [HorizontalPodAutoscaler](https://kubernetes.io/docs/tasks/run-application/horizontal-pod-autoscale-walkthrough/) 
(HPA for short) that automatically updates a workload resource (such as our deployment), 
with the aim of automatically scaling the workload to match demand.

Use the following command to create the HPA:

```bash
kubectl autoscale deployment subscription --name subscription-hpa --cpu-percent=20 --min=1 --max=3
```

You can check the current status of the new HPA by running:

```bash
kubectl get hpa
```

The output should be similar to the next:

```bash
NAME               REFERENCE                 TARGETS   MINPODS   MAXPODS   REPLICAS   AGE
subscription-hpa   Deployment/subscription   1%/20%    1         3         1          27s
```

`TARGET` column shows the average CPU consumption across all the Pods controlled by the corresponding deployment.
Current CPU consumption is about 0% as there are no clients sending requests to the server.
> 
>Note: In case the HPA instance doesn't run properly, try to install the latest Metrics Server release 
> from the `components.yaml` manifest:
> ```bash
> kubectl apply -f https://github.com/kubernetes-sigs/metrics-server/releases/latest/download/components.yaml
>```

In [12]:
!kubectl autoscale deployment subscription --name subscription-hpa --cpu-percent=20 --min=1 --max=3

horizontalpodautoscaler.autoscaling/subscription-hpa autoscaled


In [13]:
!kubectl get hpa

NAME               REFERENCE                 TARGETS              MINPODS   MAXPODS   REPLICAS   AGE
subscription-hpa   Deployment/subscription   cpu: <unknown>/20%   1         3         0          11s


## Question 8 (optional)

Run `kubectl get hpa subscription-hpa --watch` command to monitor how the autoscaler performs. 
Within a minute or so, you should see the higher CPU load; and then - more replicas. 
What was the maximum amount of the replicas during this test?


* 1
* 2
* 3
* 4

> Note: It may take a few minutes to stabilize the number of replicas. Since the amount of load is not controlled 
> in any way it may happen that the final number of replicas will differ from initial.

In [19]:
!kubectl get hpa subscription-hpa --watch

NAME               REFERENCE                 TARGETS              MINPODS   MAXPODS   REPLICAS   AGE
subscription-hpa   Deployment/subscription   cpu: <unknown>/20%   1         3         1          7m10s
^C


In [17]:
!top

=top - 00:54:22 up 57 min,  0 users,  load average: 1.68, 1.01, 0.76
Tasks:  68 total,   2 running,  65 sleeping,   0 stopped,   1 zombie
%Cpu(s): 12.9 us, 16.1 sy,  0.0 ni, 67.7 id,  0.0 wa,  0.0 hi,  3.2 si,  0.0 st
MiB Mem :   7929.6 total,    257.0 free,   2886.0 used,   4786.6 buff/cache
MiB Swap:      0.0 total,      0.0 free,      0.0 used.   4656.2 avail Mem 

    PID USER      PR  NI    VIRT    RES    SHR S  %CPU  %MEM     TIME+ COMMAND  
    422 codespa+  20   0   41.7g 526364  56448 S   6.2   6.5   1:29.53 node     
   5714 root      20   0 1292316  68752  47872 S   6.2   0.8   0:18.61 kube-sc+ 
      1 codespa+  20   0    1136    640    640 S   0.0   0.0   0:00.07 docker-+ 
      6 codespa+  20   0    7236   1664   1664 S   0.0   0.0   0:00.02 sleep    
     34 root      20   0   12196   3480   2560 S   0.0   0.0   0:00.00 sshd     
    171 codespa+  20   0    2616   1408   1408 S   0.0   0.0   0:00.00 sh       
    197 root      20   0    2616   1536   1536 S   0.0   0.0 